In [6]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

root_src = 'D:\Dogs vs. Cats'
import os
import shutil
import time
os.chdir(root_src)
import numpy as np
import h5py
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    train = model.predict_generator(train_generator, len(train_generator.filenames)/16)
    test = model.predict_generator(test_generator, len(test_generator.filenames)/16)
    print(type(train),train.shape) 
    print(type(test),test.shape) 
    with h5py.File("gap_%s.h5"%(MODEL.__name__)) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
        
start = time.time()  
write_gap(VGG16,(224,224))
write_gap(VGG19,(224,224))

end = time.time()
print('one generator time is %d s'%round(end-start))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
<class 'numpy.ndarray'> (25000, 512)
<class 'numpy.ndarray'> (12500, 512)
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
<class 'numpy.ndarray'> (25000, 512)
<class 'numpy.ndarray'> (12500, 512)
one generator time is 717 s
